In [1]:
import numpy as np

# 1. Gini Impurity Calculation
def gini_impurity(y):
    class_counts = np.bincount(y)  # Count occurrences of each class
    probabilities = class_counts / len(y)  # Calculate probabilities
    gini = 1 - np.sum(probabilities ** 2)  # Gini impurity formula
    return gini

# 2. Best Split Calculation
def best_split(X, y):
    best_gini = float('inf')  # Initialize to a large value
    best_split = None
    
    # Iterate through each feature and each possible threshold
    for feature_idx in range(X.shape[1]):
        thresholds = np.unique(X[:, feature_idx])  # Unique values of the feature
        for threshold in thresholds:
            left_mask = X[:, feature_idx] <= threshold  # Left split condition
            right_mask = ~left_mask  # Right split condition
            
            # Split the data
            left_y, right_y = y[left_mask], y[right_mask]
            
            # Calculate Gini impurity for both left and right subsets
            gini_left = gini_impurity(left_y)
            gini_right = gini_impurity(right_y)
            
            # Weighted Gini for the current split
            weighted_gini = (len(left_y) / len(y)) * gini_left + (len(right_y) / len(y)) * gini_right
            
            # Update the best split if the current one is better
            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_split = (feature_idx, threshold)
                
    return best_split

# 3. Building the Decision Tree (Recursive)
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth  # Maximum depth of the tree

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        # Stopping condition
        if len(np.unique(y)) == 1:  # Pure node
            return {'leaf': np.unique(y)[0]}
        if len(y) == 0:  # Empty node
            return {'leaf': np.random.choice(np.unique(y))}
        if self.max_depth and depth >= self.max_depth:  # Max depth reached
            return {'leaf': np.bincount(y).argmax()}  # Majority class

        # Find the best split
        feature_idx, threshold = best_split(X, y)

        # Split the data
        left_mask = X[:, feature_idx] <= threshold
        right_mask = ~left_mask
        left_X, left_y = X[left_mask], y[left_mask]
        right_X, right_y = X[right_mask], y[right_mask]

        # Recursively build the tree
        left_subtree = self._build_tree(left_X, left_y, depth + 1)
        right_subtree = self._build_tree(right_X, right_y, depth + 1)

        # Return the tree node with the split
        return {'feature': feature_idx, 'threshold': threshold, 
                'left': left_subtree, 'right': right_subtree}

    def predict(self, X):
        return [self._predict_sample(x, self.tree) for x in X]

    def _predict_sample(self, x, node):
        if 'leaf' in node:
            return node['leaf']
        if x[node['feature']] <= node['threshold']:
            return self._predict_sample(x, node['left'])
        else:
            return self._predict_sample(x, node['right'])

# 4. Example Usage
# Sample dataset: Simple binary classification
X = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])
y = np.array([0, 0, 1, 1, 1])

# Train the decision tree
tree = DecisionTree(max_depth=3)
tree.fit(X, y)

# Make predictions
predictions = tree.predict(X)
print(f"Predictions: {predictions}")


Predictions: [0, 0, 1, 1, 1]
